# High-Throughput BWA Read Mapping

This tutorial focuses on a subset of the [Data Carpentry Genomics workshop curriculum](https://datacarpentry.org/genomics-workshop/) - specifically, this page cover's how to run a BWA workflow on OSG resources. It will use the same general flow as the BWA segment of the Data Carpentry workshop with minor adjustments. The goal of this tutorial is to learn how to convert an existing BWA workflow to run on the OSPool.  


## Software Environment

An earlier version of this tutorial has you compile bwa and bring along the compiled installation. To make things a little cleaner, we are going to use a pre-built container that's now part of the tutorial. It is the `bwa.sif` file inside the `/software` directory. 

> To explore the container before using it, you need to use a terminal window (not a notebook). 
> In the terminal, make sure you are in the `tutorial-bwa` directory. Then run this command:
> 
>     $ apptainer shell software/bwa.sif
> 
> Your prompt should change to something like `Apptainer>`. From here, you can run the `bwa` command, which should look something like this: 
> 
>      Apptainer> bwa
>      Program: bwa (alignment via Burrows-Wheeler transformation)
>        Version: 0.7.17-r1198-dirty
>        Contact: Heng Li <hli@ds.dfci.harvard.edu>
>
>        Usage:   bwa <command> [options]
>
>        Command: index         index sequences in the FASTA format
>                 mem           BWA-MEM algorithm
>                 fastmap       identify super-maximal exact matches
>                 pemerge       merge overlapping paired ends (EXPERIMENTAL)
>                 aln           gapped/ungapped alignment
>                 samse         generate alignment (single ended)
>                 sampe         generate alignment (paired ended)
>                 bwasw         BWA-SW for long queries (DEPRECATED)
>
>                 shm           manage indices in shared memory
>                 fa2pac        convert FASTA to PAC format
>                 pac2bwt       generate BWT from PAC
>                 pac2bwtgen    alternative algorithm for generating BWT
>                 bwtupdate     update .bwt to the new format
>                 bwt2sa        generate SA from BWT and Occ
>
>        Note: To use BWA, you need to first index the genome with `bwa index'.
>              There are three alignment algorithms in BWA: `mem', `bwasw', and
>              `aln/samse/sampe'. If you are not sure which to use, try `bwa mem'
>              first. Please `man ./bwa.1' for the manual.
>
> 
> To exit the container, type `exit`. 

## Download Data to Analyze

Now that we have access to BWA, we need to download data to analyze. For this tutorial, we will be downloading data used in the Data Carpentry workshop. This data includes both the genome of Escherichia coli (E. coli) and paired-end RNA sequencing reads obtained from a study carried out by Blount et al. published in [PNAS](http://www.pnas.org/content/105/23/7899). Additional information about how the data was modified in preparation for this analysis can be found on the [Data Carpentry's workshop website](https://datacarpentry.org/wrangling-genomics/aio.html).

In [1]:
./download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1343k  100 1343k    0     0  2300k      0 --:--:-- --:--:-- --:--:-- 2300k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  109M  100  109M    0     0  18.0M      0  0:00:06  0:00:06 --:--:-- 26.1M
sub/
sub/SRR2584866_1.trim.sub.fastq
sub/SRR2589044_1.trim.sub.fastq
sub/SRR2589044_2.trim.sub.fastq
sub/SRR2584863_2.trim.sub.fastq
sub/SRR2584866_2.trim.sub.fastq
sub/SRR2584863_1.trim.sub.fastq


Investigating the size of the downloaded genome by typing:

In [93]:
ls -lh data/ref_genome/

total 1.4M
-rw-r--r-- 1 jovyan users 1.4M Mar 27 01:20 ecoli_rel606.fasta.gz


reveals the file is 1.4 MB. We should also check the trimmed fastq paired-end read files: 

In [94]:
ls -lh data/trimmed_fastq_small

total 332M
-rw-r--r-- 1 jovyan users  33 Mar 27 02:01 samples.txt
-rw-r--r-- 1 jovyan users 58M Jul  6  2018 SRR2584863_1.trim.sub.fastq
-rw-r--r-- 1 jovyan users 53M Jul  6  2018 SRR2584863_2.trim.sub.fastq
-rw-r--r-- 1 jovyan users 55M Jul  6  2018 SRR2584866_1.trim.sub.fastq
-rw-r--r-- 1 jovyan users 57M Jul  6  2018 SRR2584866_2.trim.sub.fastq
-rw-r--r-- 1 jovyan users 58M Jul  6  2018 SRR2589044_1.trim.sub.fastq
-rw-r--r-- 1 jovyan users 53M Jul  6  2018 SRR2589044_2.trim.sub.fastq


## Run a Single Test Job

Now that we have all items in our analysis ready, it is time to submit a single test job to map our RNA reads to the E. coli genome. 
For a single test job, we will choose a single sample to analyze. In the following example, we will align both the 
forward and reverse reads of SRR2584863 to the E. coli genome.  Our executable looks like this: 

In [95]:
cat bwa-container.sh

#!/bin/bash
# Script name: bwa-container.sh

# set bwa location
export PATH=/bwa:$PATH

echo "Indexing E. coli genome"
bwa index ecoli_rel606.fasta.gz

echo "Starting bwa alignment for SRR2584863"
bwa mem ecoli_rel606.fasta.gz SRR2584863_1.trim.sub.fastq SRR2584863_2.trim.sub.fastq > SRR2584863.aligned.sam

echo "Done with bwa alignment for SRR2584863!"

echo "Cleaning up files generated from genome indexing"
rm ecoli_rel606.fasta.gz.amb
rm ecoli_rel606.fasta.gz.ann
rm ecoli_rel606.fasta.gz.bwt
rm ecoli_rel606.fasta.gz.pac
rm ecoli_rel606.fasta.gz.sa


Putting all the pieces together, we want to edit the following submit file: 

In [96]:
cat bwa-container.sub

universe        = container
container_image = ./bwa.sif

executable  = bwa-container.sh
# arguments = 

# need to transfer bwa.tar.gz file, the reference
# genome, and the trimmed fastq files
transfer_input_files = data/ref_genome/ecoli_rel606.fasta.gz, data/trimmed_fastq_small/SRR2584863_1.trim.sub.fastq, data/trimmed_fastq_small/SRR2584863_2.trim.sub.fastq, software/bwa.sif
should_transfer_files = YES
when_to_transfer_output = ON_EXIT

log         = logs/bwa_test_job.log
output      = logs/bwa_test_job.out
error       = logs/bwa_test_job.error

+JobDurationCategory = "Medium"
request_cpus    = 1
request_memory  = 1GB
request_disk    = 1GB

queue 1


To look like this: 

```
universe    = container
container_image = software/bwa.sif

executable  = bwa-container.sh
# arguments = 

# need to transfer bwa.tar.gz file, the reference
# genome, and the trimmed fastq files
transfer_input_files = data/ref_genome/ecoli_rel606.fasta.gz, data/trimmed_fastq_small/SRR2584863_1.trim.sub.fastq, data/trimmed_fastq_small/SRR2584863_2.trim.sub.fastq, software/bwa.sif
should_transfer_files = YES
when_to_transfer_output = ON_EXIT

log         = logs/bwa_test_job.log
output      = logs/bwa_test_job.out
error       = logs/bwa_test_job.error

+JobDurationCategory = "Medium"
request_cpus    = 1
request_memory  = 1GB
request_disk    = 1GB

queue 1
```
You will notice that the .log, .out, and .error files will be saved to a folder called `logs`. 
This folder needs to be created before jobs are submitted. 

We can submit this single test job to HTCondor by typing: 

In [97]:
condor_submit bwa-container.sub

Submitting job(s).
1 job(s) submitted to cluster 13.


To check the status of the job, we can use: 

In [99]:
condor_q



-- Schedd: jovyan@jupyter-email-3achristinakconnect-40gmail-2ecom : <127.0.0.1:9618?... @ 03/27/23 02:16:39
OWNER BATCH_NAME      SUBMITTED   DONE   RUN    IDLE   HOLD  TOTAL JOB_IDS

Total for query: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for all users: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended



Upon the completion of the test job, we should investigate the output to ensure that it is what we expected and 
also review the `.log` file to help optimize future resource requests in preparation for scaling up. 


In [100]:
ls -lh *.sam

-rw-r--r-- 1 jovyan users 127M Mar 27 02:16 SRR2584863.aligned.sam


In [101]:
tail logs/bwa_test_job.error

[M::mem_pestat] (25, 50, 75) percentile: (430, 944, 2880)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 7780)
[M::mem_pestat] mean and std.dev: (1672.06, 1727.12)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 10230)
[M::mem_pestat] skip orientation FF
[M::mem_pestat] skip orientation RR
[M::mem_process_seqs] Processed 46340 reads in 2.550 CPU sec, 2.474 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem ecoli_rel606.fasta.gz SRR2584863_1.trim.sub.fastq SRR2584863_2.trim.sub.fastq
[main] Real time: 17.930 sec; CPU: 18.382 sec


In [102]:
cat logs/bwa_test_job.out

Indexing E. coli genome
Starting bwa alignment for SRR2584863
Done with bwa alignment for SRR2584863!
Cleaning up files generated from genome indexing


In [103]:
tail logs/bwa_test_job.log

	834325312  -  Run Bytes Received By Job
	132799032  -  Total Bytes Sent By Job
	834325312  -  Total Bytes Received By Job
	Partitionable Resources :     Usage  Request Allocated 
	   Cpus                 :      0.92        1         1 
	   Disk (KB)            : 944473     1048576   1107033 
	   Memory (MB)          :    143        1024      1024 

	Job terminated of its own accord at 2023-03-27T02:16:27Z with exit-code 0.
...


Here we see that we used less memory than we requested. In future jobs, we should 
request a smaller amount of memory. Prior to scaling up our analysis, 
we should run additional test jobs using these resource requests to ensure that they are sufficient to allow our job to 
complete successfully.

## Modularizing and Organizing the Job

Now that things are running, we want to make this job more modular and re-organize the files a little bit. First, we want 
to modify our executable (the shell script) so it can take in different files. We'll do this by adding an argument 
to the script. We'll modify the shell script to look like this: 

```
#!/bin/bash
# Script name: bwa-container.sh

SAMPLE=$1

# set BWA location
export PATH=/bwa:$PATH

echo "Indexing E. coli genome"
bwa index ecoli_rel606.fasta.gz

echo "Starting bwa alignment for ${SAMPLE}"
bwa mem ecoli_rel606.fasta.gz ${SAMPLE}_1.trim.sub.fastq ${SAMPLE}_2.trim.sub.fastq > ${SAMPLE}.aligned.sam

echo "Done with bwa alignment for ${SAMPLE}!"

echo "Cleaning up files generated from genome indexing"
rm ecoli_rel606.fasta.gz.amb
rm ecoli_rel606.fasta.gz.ann
rm ecoli_rel606.fasta.gz.bwt
rm ecoli_rel606.fasta.gz.pac
rm ecoli_rel606.fasta.gz.sa
```

And in the submit file, uncomment the `arguments` line and provide the sample number: 

```
arguments = SRR2584863
```

Secondly, we'll want to tidy up where our results are going. Let's say we want to 
have all of our aligned `.sam` files in a folder called `results`. We can make the folder: 

In [104]:
mkdir results

To store the aligned sequencing files in the `results` folder, we can add the `transfer_output_remaps` feature to our submit file. This feature allows us to specify a name and a path to save our output files in the format of "file1 = path/to/save/file2", where file1 is the origional name of the document and file2 is the name that we want to save the file using. In the example above, we do not change the name of the resulting output files. This feature also helps us keep an organized working space, rather than having all of our resulting sequencing files be saved to our `/home` directory. 
```
transfer_output_remaps = "SRR2584863.aligned.sam=results/SRR2584863.aligned.sam"
```

In the end, our submit file and executable should look like this: 

Executable: 

```
#!/bin/bash
# Script name: bwa-container.sh

SAMPLE=$1

echo "Indexing E. coli genome"
bwa index ecoli_rel606.fasta.gz

echo "Starting bwa alignment for ${SAMPLE}"
bwa mem ecoli_rel606.fasta.gz ${SAMPLE}_1.trim.sub.fastq ${SAMPLE}_2.trim.sub.fastq > ${SAMPLE}.aligned.sam

echo "Done with bwa alignment for ${SAMPLE}!"

echo "Cleaning up files generated from genome indexing"
rm ecoli_rel606.fasta.gz.amb
rm ecoli_rel606.fasta.gz.ann
rm ecoli_rel606.fasta.gz.bwt
rm ecoli_rel606.fasta.gz.pac
rm ecoli_rel606.fasta.gz.sa
```

Submit file: 

```
universe    = container
container_image = software/bwa.sif

executable  = bwa-container.sh
arguments = SRR2584863

# need to transfer bwa.tar.gz file, the reference
# genome, and the trimmed fastq files
transfer_input_files = data/ref_genome/ecoli_rel606.fasta.gz, data/trimmed_fastq_small/SRR2584863_1.trim.sub.fastq, data/trimmed_fastq_small/SRR2584863_2.trim.sub.fastq, software/bwa.sif
transfer_output_remaps = "SRR2584863.aligned.sam=results/SRR2584863.aligned.sam"
should_transfer_files = YES
when_to_transfer_output = ON_EXIT

log         = logs/bwa_SRR2584863_job.log
output      = logs/bwa_SRR2584863_job.out
error       = logs/bwa_SRR2584863_job.error

+JobDurationCategory = "Medium"
request_cpus    = 1
request_memory  = 500MB
request_disk    = 500MB

queue 1
```

We'll want to submit this and then double-check that our changes worked. 

In [105]:
condor_submit bwa-container.sub

Submitting job(s).
1 job(s) submitted to cluster 14.


In [107]:
condor_q



-- Schedd: jovyan@jupyter-email-3achristinakconnect-40gmail-2ecom : <127.0.0.1:9618?... @ 03/27/23 02:18:50
OWNER BATCH_NAME      SUBMITTED   DONE   RUN    IDLE   HOLD  TOTAL JOB_IDS

Total for query: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for all users: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended



In [109]:
ls -lh results/*.sam

-rw-r--r-- 1 jovyan users 127M Mar 27 02:18 results/SRR2584863.aligned.sam


## Scaling Up

In preparation for scaling up, please review our [guide on how to scale up after a successful test job](https://support.opensciencegrid.org/support/solutions/articles/12000076552-scaling-up-after-success-with-test-jobs) and how to 
[easily submit multiple jobs with a single submit file](https://support.opensciencegrid.org/support/solutions/articles/12000073165-easily-submit-multiple-jobs).

After reviewing how to submit multiple jobs with a single submit file, it is possible to determine that the most appropriate way to submit multiple jobs for this analysis is to use `queue <var> from <list.txt>`. 

To use this option, we first need to create a file with just the sample names/IDs that we want to analyze. To do this, we want to cut all information after the "_" symbol to remove the forward/reverse read information and file extensions. For example, we want SRR2584863_1.trim.sub.fastq to become just SRR2584863. 

We will save the sample names in a file called `samples.txt`:

In [110]:
cd data/trimmed_fastq_small/
ls *.fastq | cut -f 1 -d '_' | uniq > samples.txt
cd ../../

Now, we can create a new submit file called `bwa-alignment.sub` to queue a new job for each sample. 
To make it simpler to start, you can copy the `bwa-container.sub` file and modify it. 

In [124]:
cp bwa-container.sub bwa-alignment.sub

The main changes to make to the submit file are replacing each occurence of the sample 
identifier with the `$(sample)` variable, and then iterating through our list of samples 
as shown in the `queue` statement at the end. 

```
universe    = container
container_image = ./bwa.sif

executable  = bwa-container.sh
arguments = $(sample)

# need to transfer bwa.tar.gz file, the reference
# genome, and the trimmed fastq files
transfer_input_files = data/ref_genome/ecoli_rel606.fasta.gz, data/trimmed_fastq_small/$(sample)_1.trim.sub.fastq, data/trimmed_fastq_small/$(sample)_2.trim.sub.fastq, software/bwa.sif
transfer_output_remaps = "$(sample).aligned.sam=results/$(sample).aligned.sam"
should_transfer_files = YES
when_to_transfer_output = ON_EXIT

log         = logs/bwa_$(sample)_job.log
output      = logs/bwa_$(sample)_job.out
error       = logs/bwa_$(sample)_job.error

+JobDurationCategory = "Medium"
request_cpus    = 1 
request_memory  = 0.5GB
request_disk    = 1GB

queue sample from data/trimmed_fastq_small/samples.txt
```

We can then submit the jobs and should see three jobs in the queue, one for each pairs of files. 

In [125]:
condor_submit bwa-alignment.sub

Submitting job(s)...
3 job(s) submitted to cluster 16.


In [134]:
condor_q



-- Schedd: jovyan@jupyter-email-3achristinakconnect-40gmail-2ecom : <127.0.0.1:9618?... @ 03/27/23 02:23:57
OWNER BATCH_NAME      SUBMITTED   DONE   RUN    IDLE   HOLD  TOTAL JOB_IDS

Total for query: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for all users: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended



In [135]:
ls -lh results

total 382M
-rw-r--r-- 1 jovyan users 127M Mar 27 02:23 SRR2584863.aligned.sam
-rw-r--r-- 1 jovyan users 128M Mar 27 02:23 SRR2584866.aligned.sam
-rw-r--r-- 1 jovyan users 128M Mar 27 02:23 SRR2589044.aligned.sam
